In [8]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


## Prompt Templates

Most LLM applications do not pass user input directly into an LLM. Usually they will add the user input to a larger piece of text, called a prompt template, that provides additional context on the specific task at hand.

PromptTemplates help with bundling up all the logic for going from user input into a fully formatted prompt.


In [9]:
from langchain.prompts import PromptTemplate

llm_template = PromptTemplate.from_template("Tell me a joke about {topic}")
llm_prompt = llm_template.format(topic="programming")
llm_prompt

'Tell me a joke about programming'

PromptTemplates can produce a list of messages. The prompt not only contains information about the content, but also each message (its role, its position in the list, etc.). Here, what happens most often is a `ChatPromptTemplate` is a list of `ChatMessageTemplates.` Each `ChatMessageTemplate` contains instructions for how to format that `ChatMessage` - its role, and then also its content.

In [10]:
from langchain.prompts.chat import ChatPromptTemplate

system_template = "You are a helpful assistant that tells jokes about topic given by human. Tell the jokes in {joke_language} language."
human_template = "{human_text}"

chat_template = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template),
])

chat_prompt = chat_template.format_messages(joke_language="Polish", human_text="I love programming.")
chat_prompt

[SystemMessage(content='You are a helpful assistant that tells jokes about topic given by human. Tell the jokes in Polish language.'),
 HumanMessage(content='I love programming.')]

## LLM / Chat model

There are two types of language models:

`LLM`: underlying model takes a string as input and returns a string
`ChatModel`: underlying model takes a list of messages as input and returns a message

In [11]:
from langchain.llms import OpenAI

llm = OpenAI()
llm.invoke(llm_prompt)

"\n\nQ: Why did the programmer quit his job?\nA: Because he didn't get arrays."

In [12]:
from langchain.chat_models import ChatOpenAI

chat_model = ChatOpenAI()
chat_model.invoke(chat_prompt)

AIMessage(content='Świetnie! To mam dla Ciebie żart programistyczny:\n\nProgramista wraca z zakupów do domu i mówi żonie: "Kochanie, kupiłem piwo, bo wiem, że to Twoja ulubiona metoda sortowania!"\n\nŻona na to: "A gdzie jest reszta zakupów?"\n\nProgramista odpowiada: "No wiesz, sortowanie bąbelkowe jest dość wolne..."')

The base message interface is defined by BaseMessage, which has two required attributes:

- `content`: The content of the message. Usually a string.
- `role`: The entity from which the BaseMessage is coming.


LangChain provides several objects to easily distinguish between different roles:

- `HumanMessage`: A BaseMessage coming from a human/user.
- `AIMessage`: A BaseMessage coming from an AI/assistant.
- `SystemMessage`: A BaseMessage coming from the system.
- `FunctionMessage / ToolMessage`: A BaseMessage containing the output of a function or tool call. (*)

## Output parsers

OutputParsers convert the raw output of a language model into a format that can be used downstream.

In [13]:
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser

class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")
    
parser = PydanticOutputParser(pydantic_object=Joke)

system_template = "You are a helpful assistant that tells jokes about topic given by human. Tell the jokes in {joke_language} language. {format_instructions}"
human_template = "{human_text}"

chat_template = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template),
])

chat_prompt = chat_template.format_messages(joke_language="Polish", human_text="I love programming.", format_instructions=parser.get_format_instructions())
chat_prompt

[SystemMessage(content='You are a helpful assistant that tells jokes about topic given by human. Tell the jokes in Polish language. The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"setup": {"description": "question to set up a joke", "title": "Setup", "type": "string"}, "punchline": {"description": "answer to resolve the joke", "title": "Punchline", "type": "string"}}, "required": ["setup", "punchline"]}\n```'),
 HumanMessage(content='I love programming.')]

In [14]:
raw_output = chat_model.invoke(chat_prompt)
parser.invoke(raw_output)

Joke(setup='Dlaczego programiści lubią pływać?', punchline='Bo w wodzie czują się jak w float')